In [1]:
import pandas as pd
import numpy as np
import glob

import os
import tensorflow as tf   
from sklearn.preprocessing import scale
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
tf.__version__

'2.0.1'

In [4]:
tf.test.is_gpu_available()

True

In [5]:
a = 6378137
e = 8.1819190842622e-2
asq = np.power(a,2)
esq = np.power(e,2)
def ecef2lla(ecef):
    x = ecef[0]
    y = ecef[1]
    z = ecef[2]
    r = ecef[3]
    b = np.sqrt( asq * (1-esq) )
    bsq = np.power(b,2)
    ep = np.sqrt( (asq - bsq)/bsq)
    p = np.sqrt(np.power(x,2) + np.power(y,2) )
    th = np.arctan2(a*z, b*p)
    lon = np.arctan2(y,x)
    lat = np.arctan2( (z + np.power(ep,2)*b*np.power(np.sin(th),3) ), (p - esq*a*np.power(np.cos(th),3)) )
    N = a/( np.sqrt(1-esq*np.power(np.sin(lat),2)) )
    alt = p / np.cos(lat) - N
    lon = lon % (2*np.pi)
    ret = [lat, lon, alt , r]
    return ret

In [6]:
datas = glob.glob('transfer_gps/*/*/*.txt')
train_data = []
train_label = [] 
for data in datas:
    f = np.loadtxt(data,delimiter=",").copy()
    o = np.argsort(f,axis=(0))[:,3]
    f = np.array([ecef2lla(d) for d in f])
    f = f/np.array([(2*np.pi),(2*np.pi),1e+8,1e+14])
    f = f[o]
    f.resize((10,4))
    train_data.append(f) #/100000000
    if data.split('/')[1] == "indoor":
        train_label.append("indoor"+data.split('/')[-2])
    else:
        train_label.append("outdoor"+data.split('/')[-2])
        

train_data = np.array(train_data).astype('float32')
#train_data[:,:,0] -= train_data[:,:,0].mean()
#train_data[:,:,0] /= train_data[:,:,0].var()

#train_data[:,:,1] -= train_data[:,:,1].mean()
#train_data[:,:,1] /= train_data[:,:,1].var()
#train_data[:,:,2] -= train_data[:,:,2].mean()
#train_data[:,:,2] /= train_data[:,:,2].var()
#train_data[:,:,3] -= train_data[:,:,3].mean()
#train_data[:,:,3] /= 1000000
#train_label = pd.get_dummies(train_label).values.argmax(1)
#train_label = train_label.reshape(len(train_label),1)
train_label = np.array(pd.get_dummies(train_label)).astype('float32')

train_val_split = np.random.rand(len(train_data)) < 0.70
train_x = train_data[train_val_split]
train_y = train_label[train_val_split]
val_x = train_data[~train_val_split]
val_y = train_label[~train_val_split]

In [7]:
#pd.get_dummies(train_label)

In [8]:
train_label[10]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)

In [9]:
train_data[:,:,0].mean(),train_data[:,:,0].var(),train_data[:,:,0].max(),train_data[:,:,0].min()

(0.014776827, 0.005121139, 0.15694372, -0.11999485)

In [10]:
train_data[:,:,1].mean(),train_data[:,:,1].var(),train_data[:,:,1].max(),train_data[:,:,1].min()

(0.19140744, 0.02592802, 0.9196118, 0.0)

In [11]:
train_data[:,:,2].mean(),train_data[:,:,2].var(),train_data[:,:,2].max(),train_data[:,:,2].min()

(0.14311595, 0.008440535, 0.2070294, 0.0)

In [12]:
train_data[:,:,3].mean(),train_data[:,:,3].var(),train_data[:,:,3].max(),train_data[:,:,3].min()

(0.0012082175, 0.0004162902, 0.34617165, 0.0)

In [13]:
train_data.mean(),train_data.var()

(0.08762709, 0.016632685)

In [14]:
train_x.shape,train_y.shape,len(val_x)#.shape

((3170, 10, 4), (3170, 18), 1353)

In [15]:
BUFFER_SIZE = train_x.shape[0]
BATCH_SIZE = train_x.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valid_dataset = tf.data.Dataset.from_tensor_slices((val_x,val_y)).batch(len(val_x))

In [16]:
class DenseTranspose(tf.keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        self.dense = dense
        self.activation = tf.keras.activations.get(activation)
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name='bias',shape=[self.dense.input_shape[-1]],initializer="zeros")
        super().build(batch_input_shape)
    def call(self, inputs):
        z = tf.matmul(inputs, self.dense.weights[0], transpose_b = True)
        return self.activation(z + self.biases)

In [17]:
#mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0","/gpu:1"])
#with mirrored_strategy.scope():
input_o = tf.keras.layers.Input(shape=(4), name='input_layer')
dense1 = layers.Dense(4, activation='relu')
dense2 = layers.Dense(4, activation='relu')
dense3 = layers.Dense(4, activation='relu')
dense4 = layers.Dense(4, activation='relu')
models = dense1(input_o )
models = dense2(models)
models = dense3(models)
models = dense4(models)
model = tf.keras.Model(inputs=input_o, outputs=models)

input_oo = tf.keras.layers.Input(shape=(4), name='input_layer0')
models2 = DenseTranspose(dense4, activation = 'relu')(input_oo)
models2 = DenseTranspose(dense3, activation = 'relu')(models2)
models2 = DenseTranspose(dense2, activation = 'relu')(models2)
models2 = DenseTranspose(dense1, activation = 'relu')(models2)
model2 = tf.keras.Model(inputs=input_oo, outputs=models2)

input = tf.keras.layers.Input(shape=(10,4), name='input_layer1')
input1 = layers.Lambda(lambda x: x[:,0,:], output_shape=(1))(input)
input2 = layers.Lambda(lambda x: x[:,1,:], output_shape=(1))(input)
input3 = layers.Lambda(lambda x: x[:,2,:], output_shape=(1))(input)
input4 = layers.Lambda(lambda x: x[:,3,:], output_shape=(1))(input)
input5 = layers.Lambda(lambda x: x[:,4,:], output_shape=(1))(input)
input6 = layers.Lambda(lambda x: x[:,5,:], output_shape=(1))(input)
input7 = layers.Lambda(lambda x: x[:,6,:], output_shape=(1))(input)
input8 = layers.Lambda(lambda x: x[:,7,:], output_shape=(1))(input)
input9 = layers.Lambda(lambda x: x[:,8,:], output_shape=(1))(input)
input10= layers.Lambda(lambda x: x[:,9,:], output_shape=(1))(input)
model_1 = model(input1)
model_2 = model(input2)
model_3 = model(input3)
model_4 = model(input4)
model_5 = model(input5)
model_6 = model(input6)
model_7 = model(input7)
model_8 = model(input8)
model_9 = model(input9)
model_10= model(input10)
merge_layer = tf.keras.layers.concatenate(inputs=[model_1, model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_9,model_10])
dense5 = layers.Dense(32, activation='relu')
dense6 = layers.Dense(24, activation='relu')
dense7 = layers.Dense(16, activation='relu')
model_encoder = dense5(merge_layer)
model_encoder = dense6(model_encoder)
model_encoder = dense7(model_encoder)
model_down = tf.keras.Model(inputs=[input], outputs=model_encoder,name = "encoder")#input1, input2,input3,input4,input5,input6,input7,input8,input9,input10


input_decoder = tf.keras.layers.Input(shape=(16), name='input_layer2')#model_down(input_encoder)
model_decoder = DenseTranspose(dense7, activation = 'relu')(input_decoder)
model_decoder = DenseTranspose(dense6, activation = 'relu')(model_decoder)
model_decoder = DenseTranspose(dense5, activation = 'relu')(model_decoder)
model_decoder = layers.Reshape((10,4))(model_decoder)
"""
out1 = layers.Lambda(lambda x: x[:,0,:], output_shape=(1))(model_decoder)
out2 = layers.Lambda(lambda x: x[:,1,:], output_shape=(1))(model_decoder)
out3 = layers.Lambda(lambda x: x[:,2,:], output_shape=(1))(model_decoder)
out4 = layers.Lambda(lambda x: x[:,3,:], output_shape=(1))(model_decoder)
out5 = layers.Lambda(lambda x: x[:,4,:], output_shape=(1))(model_decoder)
out6 = layers.Lambda(lambda x: x[:,5,:], output_shape=(1))(model_decoder)
out7 = layers.Lambda(lambda x: x[:,6,:], output_shape=(1))(model_decoder)
out8 = layers.Lambda(lambda x: x[:,7,:], output_shape=(1))(model_decoder)
out9 = layers.Lambda(lambda x: x[:,8,:], output_shape=(1))(model_decoder)
out10= layers.Lambda(lambda x: x[:,9,:], output_shape=(1))(model_decoder)
model_1 = model2(out1)
model_2 = model2(out2)
model_3 = model2(out3)
model_4 = model2(out4)
model_5 = model2(out5)
model_6 = model2(out6)
model_7 = model2(out7)
model_8 = model2(out8)
model_9 = model2(out9)
model_10= model2(out10)
merge_layer_out = tf.keras.layers.concatenate(inputs=[model_1, model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_9,model_10])
merge_layer_out = layers.Reshape((10,4))(merge_layer_out)
"""
model_up = tf.keras.Model(inputs=[input_decoder], outputs=[model_decoder],name = "decoder")
#model_decoder2 = layers.Reshape((10,4))(merge_layer_out)

input_ann = tf.keras.layers.Input(shape=(16), name='input_layer3')
model_ann = tf.keras.layers.Dense(24, activation='relu')(input_ann)
model_ann = tf.keras.layers.Dropout(0.5, noise_shape=None, seed=None)(model_ann)
model_ann = tf.keras.layers.Dense(32, activation='relu')(model_ann)
model_ann = tf.keras.layers.Dropout(0.5, noise_shape=None, seed=None)(model_ann)
model_ann = tf.keras.layers.Dense(32, activation='relu')(model_ann)
model_ann = tf.keras.layers.Dropout(0.5, noise_shape=None, seed=None)(model_ann)
model_ann = tf.keras.layers.Dense(32, activation='relu')(model_ann)
model_ann = tf.keras.layers.Dense(18, activation='softmax')(model_ann)
model_ANN = tf.keras.Model(inputs=[input_ann], outputs=model_ann,name = "ann")


input_full = tf.keras.layers.Input(shape=(10,4), name='input_layer4')
encoder_out = model_down(input_full)
decoder_out = model_up(encoder_out)
ann_out = model_ANN(encoder_out)
model_encoder_decoder = tf.keras.Model(inputs=[input_full],outputs=[decoder_out,ann_out],name = 'encoder_decoder_ann')




#model_encoder_decoder.summary()
#optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2, momentum=1e-5)
#model_encoder_decoder.compile(optimizer = 'sgd', loss=losses.MeanSquaredError())

In [18]:
shared_vars = model_encoder_decoder.trainable_variables[:14]
decode_vars = model_encoder_decoder.trainable_variables[14:21]
ann_vars = model_encoder_decoder.trainable_variables[21:]

In [19]:
def model_loss(output,t_x,t_y):
    #print("in loss")
    output_AE , output_label = output
    #output_AE = output
    mse = losses.MeanSquaredError()
    AE_loss = mse(t_x,output_AE)
    ann_loss =losses.categorical_crossentropy(t_y,output_label)
    total_loss = AE_loss*100 + ann_loss
    
    return AE_loss,ann_loss

In [20]:
learning_rate_A = tf.keras.optimizers.schedules.ExponentialDecay(
initial_learning_rate=1e-2, decay_steps=100, decay_rate=0.9)
optimizer_A = tf.optimizers.SGD(learning_rate=learning_rate_A , momentum=1e-5)
learning_rate_B = tf.keras.optimizers.schedules.ExponentialDecay(
initial_learning_rate=1e-2, decay_steps=100, decay_rate=0.9)
optimizer_B = tf.optimizers.Adam(learning_rate=learning_rate_B)

In [21]:
def train_step(t_x,t_y):
  
    with tf.GradientTape() as AE_tape,tf.GradientTape() as ANN_tape:
        output = model_encoder_decoder(t_x, training=True)
        #AE_loss = model_loss(output,t_o,t_y)
        AE_loss,ANN_loss = model_loss(output,t_x,t_y)
        #gradients = tape.gradient(total_loss, shared_vars+decode_vars)
        #optimizer_A.apply_gradients(zip(gradients, shared_vars+decode_vars))

    if np.random.rand() < 0.5:
        #print("AE")
        gradients_AE = AE_tape.gradient(AE_loss, shared_vars+decode_vars)
        #gradients_AE = [tf.clip_by_value(g, -1,1) for g in gradients_AE]

        #print("AE gradient : ",gradients_AE)
        optimizer_A.apply_gradients(zip(gradients_AE, shared_vars+decode_vars))
    
    else:
        #print("ANN")
        gradients_ANN = ANN_tape.gradient(ANN_loss, shared_vars+ann_vars)
        #print("ANN gradient : ",gradients_ANN)
        #gradients_ANN = [tf.clip_by_value(g, -1,1) for g in gradients_ANN] 
        optimizer_B.apply_gradients(zip(gradients_ANN, shared_vars+ann_vars))
    
    return np.array(AE_loss).mean(),np.array(ANN_loss).mean()

In [22]:
checkpoint_dir = './training_checkpoints_lla_model_1206'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_1206_model_{epoch}")
checkpoint = tf.train.Checkpoint(optimizerA=optimizer_A,
                                 optimizerB=optimizer_B,
                                 model_encoder_decoder=model_encoder_decoder,
                                 model_down=model_down)

In [23]:
def validation(v_x,v_y):
    output = model_encoder_decoder(v_x)
    #output_AE = output
    output_AE , output_label = output
    mse = losses.MeanSquaredError()
    AE_loss = mse(v_x,output_AE)
    ann_loss = losses.categorical_crossentropy(v_y,output_label)
    total_loss = AE_loss*100 + ann_loss
    #print(output_label[200])
    #print(v_y[200])
    #print(ann_loss)
    #print("AE loss : {},".format(np.array(AE_loss).mean()))
    print("AE loss : {}, ANN loss : {}, Total loss : {}".format(np.array(AE_loss).mean(),np.array(ann_loss).mean(),np.array(total_loss).mean()))

In [24]:
import time
def train(epochs):
  for epoch in range(epochs):
    start = time.time()
    all_AE = []
    all_ANN =[]
    for x,y in train_dataset:
        #AE_loss = train_step(x,y)
        AE_loss,ANN_loss = train_step(x,y)
        all_AE.append(AE_loss)
        all_ANN.append(ANN_loss)
    #print("train AE loss : {}".format(np.array(all_AE).mean()))
    print("train AE loss : {}, train ANN loss : {}".format(np.array(all_AE).mean(),np.array(all_ANN).mean()))
    validation(val_x,val_y)
    checkpoint.save(file_prefix=checkpoint_prefix)
    print("learning rate A : ",optimizer_A._decayed_lr(tf.float32))
    print("learning rate B : ",optimizer_B._decayed_lr(tf.float32))
    print(f'Time for epoch {epoch + 1} is {time.time() - start:.4f} sec')

In [25]:
train(500)

train AE loss : 0.02366378903388977, train ANN loss : 2.891064405441284
AE loss : 0.023558063432574272, ANN loss : 2.8905317783355713, Total loss : 5.246337890625
learning rate A :  tf.Tensor(0.009989469, shape=(), dtype=float32)
learning rate B :  tf.Tensor(0.01, shape=(), dtype=float32)
Time for epoch 1 is 0.4609 sec
train AE loss : 0.023620035499334335, train ANN loss : 2.8906657695770264
AE loss : 0.023513931781053543, ANN loss : 2.890537738800049, Total loss : 5.241930961608887
learning rate A :  tf.Tensor(0.00997895, shape=(), dtype=float32)
learning rate B :  tf.Tensor(0.01, shape=(), dtype=float32)
Time for epoch 2 is 0.1297 sec
train AE loss : 0.023575840517878532, train ANN loss : 2.8906660079956055
AE loss : 0.023987798020243645, ANN loss : 2.8902242183685303, Total loss : 5.289003849029541
learning rate A :  tf.Tensor(0.00997895, shape=(), dtype=float32)
learning rate B :  tf.Tensor(0.009989469, shape=(), dtype=float32)
Time for epoch 3 is 0.2060 sec
train AE loss : 0.02405

In [35]:
a={}
a.setdefault(1,[]).append([1,2])

In [36]:
a

{1: [[1, 2]]}

In [37]:
a.setdefault(1,[]).append([2,3])

In [38]:
a

{1: [[1, 2], [2, 3]]}

In [39]:
np.array(a[1])

array([[1, 2],
       [2, 3]])

In [40]:
np.linalg.norm(a[1],axis=1,keepdims=True)

array([[2.23606798],
       [3.60555128]])

In [41]:
np.array(a[1])/np.linalg.norm(a[1],axis=1,keepdims=True)

array([[0.4472136 , 0.89442719],
       [0.5547002 , 0.83205029]])

In [26]:
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      model_encoder_decoder.optimizer.lr.numpy()))

In [27]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

NameError: name 'decay' is not defined

In [ ]:
model_encoder_decoder.fit(train_dataset,
                epochs=500,
                validation_data=valid_dataset,
                callbacks = callbacks
                         )

In [ ]:
valid_dataset,val_x.shape

In [ ]:
val_latent = model_down(val_x)

In [ ]:
val_latent.shape

In [ ]:
from sklearn import manifold, datasets
import matplotlib.pyplot as plt

In [ ]:
train_val_split = np.random.rand(len(val_latent)) < 0.4

In [ ]:
X_tsne = manifold.TSNE(n_components=2, init='pca', n_iter=5000, method='exact').fit_transform(val_latent[train_val_split])

In [ ]:
X_tsne.shape

In [ ]:
val_latent.shape

In [ ]:
x_min, x_max = X_tsne.min(0), X_tsne.max(0)
X_norm = (X_tsne - x_min) / (x_max - x_min)
plt.figure(figsize=(20, 20))
for i in range(X_norm.shape[0]):
    plt.text(X_norm[i, 0], X_norm[i, 1], str(y[i,0]), color=plt.cm.Set1(y[i,0]), 
             fontdict={'weight': 'bold', 'size': 9})
plt.xticks([])
plt.yticks([])
plt.savefig("result_lla/pca_result_latent16_model1_2_XYZ")
plt.show()

In [ ]:
DNN = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(256, activation='relu'),
      layers.Dense(labels.shape[1], activation='softmax'),
    ])

In [ ]:
train_encoded = np.array(autoencoder.encoder(train_x))
val_encoded = np.array(autoencoder.encoder(val_x))

In [ ]:
DNN.compile(optimizer='adam', loss=losses.CategoricalCrossentropy())

In [ ]:
test_encoder = np.array(autoencoder.encoder(test_features))
y_ = DNN(test_encoder)

In [ ]:
np.argmax(np.array(y_[2]))